# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [55]:
import pandas as pd
import numpy as np
import openpyxl
import re
import json

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [48]:
# Utilizamos el parámetro 'compression' con el valor 'gzip' para indicar que el archivo está comprimido.
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true',compression='gzip')
# Mostrar el DataFrame cargado desde el archivo CSV
loans = loans.drop('Unnamed: 0',axis=1)
loans


/tmp/ipykernel_5094/2247031785.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true',compression='gzip')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,D4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,E4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,F4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [49]:
# Obtener los tipos de datos de las columnas en el DataFrame 'loans'
column_types = loans.dtypes
# Mostrar los tipos de datos de cada columna en el DataFrame
column_types

id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
funded_amnt_inv          float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 150, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [50]:
datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [51]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [52]:
# Guardar el DataFrame en formato pickle
datos_dict.to_pickle('datos_dict.pkl')

In [53]:
# Cargar el DataFrame desde el archivo pickle
datos_dict_cargado = pd.read_pickle('datos_dict.pkl')
datos_dict_cargado

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [54]:
#Busco que en la descripción venga algo que diga date.
def buscar_date(dataframe):
    features_date = set()  

    for index, row in dataframe.iterrows():
        descripcion = row['description']
        feature = row['feature']
        # Verifica que la descripción sea una cadena antes de aplicar la expresión regular
        if isinstance(descripcion, str) and re.search(r'date', descripcion, flags=re.IGNORECASE):
            # Si se encuentra la palabra "date", agrega el nombre de la feature al conjunto
            features_date.add(feature)

    return features_date
#Obtengo el set con las columas del DataFrame que deben de ser convertidas a date
dates=buscar_date(datos_dict)
#Convierto las columnas correspondientes a date
for columna in dates:
    #Me aseguro que realmente exista la columna
    if columna in loans.columns:
        loans[columna] = pd.to_datetime(loans[columna], errors='coerce', format='%Y-%m-%d')
loans.dtypes

id                                int64
member_id                       float64
loan_amnt                       float64
funded_amnt                     float64
funded_amnt_inv                 float64
                              ...      
settlement_status                object
settlement_date          datetime64[ns]
settlement_amount               float64
settlement_percentage           float64
settlement_term                 float64
Length: 150, dtype: object

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [58]:
def imputar_valores_faltantes(dataframe):
        # Elimina filas que contienen solo NaN
    dataframe = dataframe.dropna(axis=0, how='all')

    # Elimina columnas que contienen solo NaN
    dataframe = dataframe.dropna(axis=1, how='all')
    estrategia_imputacion = {}
    for columna in dataframe.columns:
        tipo_dato = dataframe[columna].dtype
        valor = 'la columna no fue modificada'
        estrategia = 'la columna no fue modificada'

        if tipo_dato == 'float64':
            # Con el fin de no cambiar la distribución de los datos
            estrategia = 'mean'
            valor = dataframe[columna].mean()
        else:
            # No tengo la información suficiente para saber que hacer con los datos
            # de tipo int,date,object,por tanto, es mejor ponerle un missing
            estrategia = 'identificador'
            valor = 'missing' 
        dataframe[columna].fillna(valor, inplace=True)
        estrategia_imputacion[columna] = {'estrategia':estrategia,'valor':valor}
    return dataframe,estrategia_imputacion
loans,estrategia_imputacion = imputar_valores_faltantes(loans)
loans

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_status,deferral_term,hardship_amount,hardship_length,disbursement_method,debt_settlement_flag,settlement_status,settlement_amount,settlement_percentage,settlement_term
0,38098114,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,MANAGEMENT,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
1,36805548,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
2,37842129,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,D1,Programming Analysis Supervisor,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
3,37612354,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,D4,Senior Sales Professional,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
4,37662224,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,C3,Technical Specialist,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,22454240,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,B1,Software Engineer,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
99996,11396920,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,C1,Assistant Professor of English,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
99997,8556176,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,E4,Facilities Tech 3,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116
99998,24023408,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,F4,Records Examiner Analyst,...,missing,3.0,110.335568,3.0,Cash,N,missing,4768.376357,47.720519,8.265116


In [59]:
estrategia_imputacion

{'id': {'estrategia': 'identificador', 'valor': 'missing'},
 'loan_amnt': {'estrategia': 'mean', 'valor': 14886.93},
 'funded_amnt': {'estrategia': 'mean', 'valor': 14886.93},
 'funded_amnt_inv': {'estrategia': 'mean', 'valor': 14883.9105},
 'term': {'estrategia': 'identificador', 'valor': 'missing'},
 'int_rate': {'estrategia': 'mean', 'valor': 13.2780734},
 'installment': {'estrategia': 'mean', 'valor': 437.3318244000001},
 'grade': {'estrategia': 'identificador', 'valor': 'missing'},
 'sub_grade': {'estrategia': 'identificador', 'valor': 'missing'},
 'emp_title': {'estrategia': 'identificador', 'valor': 'missing'},
 'emp_length': {'estrategia': 'identificador', 'valor': 'missing'},
 'home_ownership': {'estrategia': 'identificador', 'valor': 'missing'},
 'annual_inc': {'estrategia': 'mean', 'valor': 74689.2425825},
 'verification_status': {'estrategia': 'identificador', 'valor': 'missing'},
 'issue_d': {'estrategia': 'identificador', 'valor': 'missing'},
 'loan_status': {'estrategia'

### Codigo para salvar y cargar JSONs

In [61]:
#Guardamos la información en el json
with open('estrategia_imputacion.json', 'w') as json_file:
    json.dump(estrategia_imputacion, json_file)

#Cargamos el json que guardamos
with open('estrategia_imputacion.json', 'r') as json_file:
    estrategia_imputacion = json.load(json_file)
estrategia_imputacion

{'id': {'estrategia': 'identificador', 'valor': 'missing'},
 'loan_amnt': {'estrategia': 'mean', 'valor': 14886.93},
 'funded_amnt': {'estrategia': 'mean', 'valor': 14886.93},
 'funded_amnt_inv': {'estrategia': 'mean', 'valor': 14883.9105},
 'term': {'estrategia': 'identificador', 'valor': 'missing'},
 'int_rate': {'estrategia': 'mean', 'valor': 13.2780734},
 'installment': {'estrategia': 'mean', 'valor': 437.3318244000001},
 'grade': {'estrategia': 'identificador', 'valor': 'missing'},
 'sub_grade': {'estrategia': 'identificador', 'valor': 'missing'},
 'emp_title': {'estrategia': 'identificador', 'valor': 'missing'},
 'emp_length': {'estrategia': 'identificador', 'valor': 'missing'},
 'home_ownership': {'estrategia': 'identificador', 'valor': 'missing'},
 'annual_inc': {'estrategia': 'mean', 'valor': 74689.2425825},
 'verification_status': {'estrategia': 'identificador', 'valor': 'missing'},
 'issue_d': {'estrategia': 'identificador', 'valor': 'missing'},
 'loan_status': {'estrategia'